# [CW2](https://www.cs.bris.ac.uk/Teaching/Resources/COMS21202/cw2/cw2.html)
## TODO
- Import train data
- Find features
- Feature selection
- Build classifier
- Show decision boundries
- Create test data
- Classify test data

## Potential features
- Edges
- Corners
- black pixel count
- texture detection using histograms
- Number of shapes (find joined shapes) e.g. 2 shapes likely an i
- width to height ratio of cropped image

**In Fourier Domain space**:
- rectangle at corner differentiate t and s
- calculate total magnitude in arc from centre with particular radius

## Reading list
- [A Fourier Descriptor Based Character Recognition Engine](http://cs.boisestate.edu/~tim/papers/spie-jared.pdf) - includes list of features
- [Character Recognition Using Fourier Descriptors](http://cs.boisestate.edu/~amit/mscs/samples/Jared_Hopkins_Project_Report.pdf) - Very long but good detail + experiments

In [38]:
import re
import glob
import scipy.misc
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
import numpy.fft


%matplotlib inline
# notebook
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = (64.0, 48.0)


In [124]:
# Read our images

# Read image and convert to binary
def load_img(filename):
    img = scipy.misc.imread(filename, flatten=True, mode='L')
    img[img == 255] = 1
    img = np.logical_not(img).astype(int)
    return img

# Show a binary image
def show_img(img, title=None):
    fig, ax = plt.subplots()
    vmax = 255
    if np.max(img) == 1:
        vmax = 1
    ax.imshow(img, cmap='gray', vmin=0, vmax=vmax)
    if title:
        plt.suptitle(title)
    plt.show()
    
# Calc magnitude spectrum
def calc_mag_spec(img):
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    return 20*np.log(np.abs(fshift))

In [118]:
class Character():
    def __init__(self, filename, label=None, filename_id=None):
        self.filename = filename
        self.img      = load_img(filename)
        self.spec     = calc_mag_spec(self.img)
        self.label    = label
        self.filename_id = filename_id
        
        if type(self.label) == str:
            self.label = self.label.upper()
        
    def show(self):
        show_img(self.img)
        
    def show_spec(self):
        show_img(self.spec)
        
    def show_with_spec(self):
        fig = plt.figure()
        ax = fig.add_subplot(1, 2, 1)
        plt.imshow(self.img, cmap='gray', vmin = 0, vmax = 255)
        ax = fig.add_subplot(1, 2, 2)
        plt.imshow(self.spec, cmap='gray', vmin = 0, vmax = 255)
        
        plt.tight_layout()
        plt.show()
        
    def __repr__(self):
        return str(self.label)
        
class Dataset():
    def __init__(self, dataset):
        self.dataset = dataset
    
    def label_is(self, label):
        return Dataset([c for c in self.dataset if c.label == label.upper()])
    
    def show(self):        
        fig = plt.figure()
        for i, c in enumerate(self.dataset, start=1):
            ax = fig.add_subplot(len(self.dataset), 1, i)
            plt.imshow(c.img, cmap='gray', vmin = 0, vmax = 255)
            ax.axis('off')
            ax.set_title(c.filename_id)
        
        plt.tight_layout()
        plt.show()
        
    def show_with_spec(self):
        fig = plt.figure()     
        for i, c in enumerate(self.dataset):
            ax = fig.add_subplot(len(self.dataset), 2, i*2+1)
            plt.imshow(c.img, cmap='gray', vmin = 0, vmax = 255)
            ax.axis('off')
            ax.set_title(c.filename_id)
            
            ax = fig.add_subplot(len(self.dataset), 2, i*2+2)
            plt.imshow(c.spec, cmap='gray', vmin = 0, vmax = 255)
            ax.axis('off')
            ax.set_title('%s magnitude spectrum' % c.filename_id)
            
        plt.tight_layout()
        plt.subplots_adjust(right=0.3)
        plt.show()
    
    def append(self, character):
        self.dataset.append(character)
        return self
    
    def __iter__(self):
        return iter(self.dataset)
    
    def __len__(self):
        return len(self.dataset)
    
    def __contains__(self, x):
        return x in self.dataset
    
    def __setitem__(self, key, value):
        self.dataset[key] = value
        
    def __getitem__(self, key):
        if type(key) == int:
            return self.dataset[key]
        elif type(key) == str:
            if len(key) == 1:
                return self.label_is(key)
            else:
                i = self.key_by_fid(key)
                if i is not None:
                    return self.dataset[i]
                else:
                    raise IndexError('unknown key %s' % key)
        elif type(key) == slice:
            return Dataset(self.dataset[key])
        else:
            raise IndexError('unknown key %s' % key)
            
    def key_by_fid(self, filename_id):
        for i, c in enumerate(self.dataset):
            if c.filename_id == filename_id.upper():
                return i
        return None
    
    def __delitem__(self, key):
        if type(key) == str:
            key = self.key_by_fid(key)

        del self.dataset[key]    
        
    def __repr__(self):
        return "".join([c.label for c in self.dataset])
    
    

In [92]:
# Load training dataset
def load_data(path):
    dataset = Dataset([])
    for filename in glob.glob(path):
        m = re.search(r"\/([a-z])([0-9]+)\.GIF", filename, re.IGNORECASE)
        label = None
        filename_id = None
        if m:
            label =  m.group(1)
            filename_id = "%s%s" % (label, m.group(2))
        dataset.append(Character(filename, label=label, filename_id=filename_id))
    return dataset


train_dataset = load_data('characters_data/*.GIF')
# Remove A1 & B1 for now
del train_dataset['A1'] 
del train_dataset['B1']

In [130]:
# Show the spectrums
train_dataset.show_with_spec()

In [140]:
# Experiment 1, quadrant
def quadrants_slice(img):
    h, w = img.shape
    
    top_left = img[:h/2,:w/2]
    top_right = img[:h/2,w/2:]
    bot_left = img[h/2:,:w/2]
    bot_right = img[h/2:,w/2:]
    
    return [top_left, top_right, bot_left, bot_right]

def centre_slice(img, h_ratio=8):
    h, w = img.shape
    return img[h/2-h/h_ratio:h/2+h/h_ratio,:]

def center_box(img, h_ratio=8, w_ratio=8):
    h, w = img.shape
    return img[h/2-h/h_ratio:h/2+h/h_ratio,w/2-w/w_ratio:w/2+w/w_ratio]

def img_stats(img):
    mean = np.mean(img)
    print ('mean: %.3f' % (mean))

for c in train_dataset:
    quads = quadrants_slice(c.spec)
    cs = centre_slice(c.spec)
    cb = center_box(c.spec)
    cb_quads = quadrants_slice(cb)
    print '%s - cs: %.3f, cb: %.3f, cb_qs: %s, qs: %s' % (c.filename_id, np.mean(cs), np.mean(cb), str([np.mean(q) for q in cb_quads]), str([np.mean(q) for q in quads]))

S1 - cs: 64.466, cb: 88.260, cb_qs: [86.592306427798462, 89.374009543883261, 89.497791251381045, 87.575271143915245], qs: [51.864842451765604, 54.186446941812179, 54.240560816774753, 52.107552212819066]
S10 - cs: 66.018, cb: 89.258, cb_qs: [85.857790436088592, 92.067443368700125, 92.138571097074134, 86.966767543337795], qs: [53.144759837251144, 56.182230872706974, 56.231170674941808, 53.396861691787379]
S2 - cs: 60.704, cb: 84.789, cb_qs: [82.721165955088708, 86.028641721253635, 86.318920228191345, 84.085854700487388], qs: [49.16496611711765, 50.373936764398216, 50.423755782638146, 49.471283373206028]
S3 - cs: 62.554, cb: 85.668, cb_qs: [82.903367006927638, 87.819008537033113, 87.842448821831724, 84.108188488801616], qs: [49.446396551577976, 52.454912100012699, 52.493075503047685, 49.722250580278313]
S4 - cs: 60.115, cb: 85.844, cb_qs: [82.015346410249123, 88.905645751821467, 89.183847101654578, 83.269267618132773], qs: [49.533664564528635, 52.55896927596649, 52.63686992707725, 49.8565